In [25]:
import math
import numpy as np
import sys
from typing import List, Tuple
from functools import lru_cache, reduce, cmp_to_key
from itertools import starmap, accumulate, chain, islice, product, tee, pairwise, combinations
from collections import Counter, deque
from heapq import heapify, heappushpop, heappop, heappush
from operator import add, neg
from bisect import bisect_right
import matplotlib.pyplot as plt

import arrays
import trees
import lists
import graphs
import search
from importlib import reload

reload(arrays)
reload(trees)
reload(lists)
reload(graphs)
reload(search)
import arrays as ar
import trees as tr
import lists as ll
import graphs as g
from lists import Node

print()

In [28]:
n = ll.make([0, 1, 2], 1)

ll.pprint(n)

0 1 2 -> 1


In [6]:
''.join(map(str,ll.subtract_lists(
                    ll.make(list(map(int, "020055383525634518999521060086463321841"))),
                    ll.make(list(map(int, "10835173613544430215114275094755963004"))),
                )))

'9220209912090088784406784991707358837'

In [346]:
def longest_substring(s: str) -> str:
    @lru_cache(None)
    def f(i: int, j: int) -> int:
        i, j = i - 1, j - 1
        return int(i >= 0 and s[i] == s[j] and f(i, j) < (j - i) and f(i, j) + 1)

    acc = lambda a, ij: a if f(*ij) <= a[0] else (f(*ij), ij[0])
    mxl, mxi = reduce(acc, combinations(range(1, len(s) + 1), 2), (0, 0))
    return s[mxi - mxl : mxi]

In [347]:
def longest_substring(s: str) -> str:
    n = len(s)
    mx = ''
    for i in range(n-1):
        sj = j = i + len(mx) + 1
        while (sj := s.find(s[i:j], max(j, sj))) > 0:
            mx = s[i:j]
            j += 1

    return mx if mx != '' else '-1'

In [351]:
def longest_substring(s: str) -> str:
    n = len(s)
    mx = ''
    for i in range(n-1):
        j = i + len(mx) + 1
        while s[i:j] in s[j:]:
            mx = s[i:j]
            j += 1

    return mx if mx != '' else '-1'

In [352]:
longest_substring("ab12acb")

'a'

In [353]:
longest_substring("dedbb12aaacead")

'd'

In [354]:
longest_substring("teeth_for_teeth")

'geeks'

In [261]:
"abc".find("b", 1, 3)

1

In [225]:
def longest_substring(s: str) -> str:
    n = len(s)
    max = ''
    for j in range(1, n):
        i = j - len(max) - 1
        while i >= 0 and s[i:j] in s[j:n]:
            max = s[i:j]
            i -= 1

    return max if max != '' else '-1'

In [134]:
def longest_substring(s: str) -> str:
    n = len(s)
    dp = [[0]*(n+1) for _ in range(n+1)]
    mxl = mxi = 0
    for i, j in combinations(range(n), 2):
        if s[i] == s[j] and dp[i][j] < j - i:
            l = dp[i+1][j+1] = dp[i][j] + 1
            if l > mxl:
                mxl, mxi = l, i + 1
    return s[mxi - mxl : mxi]

In [137]:
"" and 1

''

In [161]:
reversed(range(3))

In [183]:
reversed("abc")